In [33]:
# Import neccessary modules and access file
import numpy as np
import matplotlib.pyplot as plt
from netCDF4 import Dataset
import metpy.calc as calc
from metpy.units import units
import pyart
from scipy.interpolate import interp2d


Data = Dataset("cm1out.nc")

dataslice = 0
time = 100

In [34]:
# Access necessary variables (DbZ, U/V, X/Y) on slice 0 of data
x = Data['xh'][:].data * 1000
y = Data['yh'][:].data * 1000

u = Data['uinterp'][time,dataslice,:,:].data
v = Data['vinterp'][time,dataslice,:,:].data
w = Data['winterp'][time,dataslice,:,:].data
ref = Data['dbz'][time,dataslice,:,:].data
prs = Data['prspert'][time,dataslice,:,:].data / 100
xvort = Data['xvort'][time,dataslice,:,:].data
yvort = Data['yvort'][time,dataslice,:,:].data
zvort = Data['zvort'][time,dataslice,:,:].data
v = Data['v10'][time,:,:].data
u = Data['u10'][time,:,:].data

# Create streamwise vort
streamvort = (xvort*u + yvort*v + zvort*w) / np.sqrt(u*u + v*v + w*w)

# Create regular 2D grid for interpolation
x_reg = np.linspace(x.min(), x.max(), 10000)
y_reg = np.linspace(y.min(), y.max(), 10000)
x_reg, y_reg = np.meshgrid(x_reg, y_reg)

# Interpolate the data onto the regular grid
f_u = interp2d(x, y, u, kind='linear')
f_v = interp2d(x, y, v, kind='linear')

u_reg = f_u(x_reg[0, :], y_reg[:, 0])
v_reg = f_v(x_reg[0, :], y_reg[:, 0])

# Calculate Wind Speed
wspd = np.sqrt(u**2 + v**2)

# Create plots with more space between subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 6))
fig.suptitle('Wind Speed + Wind Vectors + Streamwise Vort Cont. and Reflectivity + Pressure Peturb. at Surface')

# Plot vertical vorticity as colormesh overlaid by streamlines of u and v
ax1.set_title('Wind Speed + Wind Vectors + Streamwise Vort Cont.')

# Use pcolormesh for zvort
zvort_plot = ax1.pcolormesh(x, y, wspd, cmap='pyart_Theodore16', vmin=0, vmax=100)
streamcon = ax1.contour(x, y, streamvort, [0.5], colors='r')
ax2.clabel(streamcon, inline=True, fontsize=8, fmt='%.1f')

# Add colorbar for the colormesh
cbar = plt.colorbar(zvort_plot, ax=ax1, label='Wind Speed (m/s)')

# Plot streamlines with default spacing
quiv = ax1.streamplot(x_reg[0, :], y_reg[:, 0], u_reg, v_reg, color='black', density=8, linewidth=0.4, arrowsize=1)

ax1.set_xlabel('X (meters)')
ax1.set_ylabel('Y (meters)')
ax1.set_xlim([-20000, 20000])
ax1.set_ylim([-20000, 20000])

# Show max wind speed in the top left corner
max_wspd = np.max(wspd)

# Increase spacing between subplots
plt.subplots_adjust(wspace=0.4)

# Plot reflectivity with pyart_ChaseSpectral colormap and vmax=90
ax2.set_title('Reflectivity + Pressure Perturb.')
ref_plot = ax2.pcolormesh(x, y, ref, cmap='pyart_ChaseSpectral', vmin=0, vmax=90)
# Add contours of prs
prs_contour = ax2.contour(x, y, prs, colors='white', linewidths=1)
ax2.clabel(prs_contour, inline=True, fontsize=8, fmt='%.1f')  # Add labels to contours
ax2.set_xlabel('X (meters)')
ax2.set_ylabel('Y (meters)')
fig.colorbar(ref_plot, ax=ax2, label='Reflectivity (dBZ)')
ax2.set_xlim([-20000, 20000])
ax2.set_ylim([-20000, 20000])

# Show min pressure in the top left corner of ax2
min_prs = np.min(prs)

ax1.text(-19900, 19900, f'Max Wind Speed: {max_wspd:.2f} m/s', color='white', fontsize=7, bbox=dict(facecolor='black', alpha=0.5))
ax2.text(-19900, 19900, f'Min Pressure Pet.: {min_prs:.2f} hPa', color='white', fontsize=7, bbox=dict(facecolor='black', alpha=0.5))
plt.show()

C:\Users\micha\AppData\Local\Temp\ipykernel_17528\2552484690.py:27: DeprecationWarning: `interp2d` is deprecated!
`interp2d` is deprecated in SciPy 1.10 and will be removed in SciPy 1.13.0.

For legacy code, nearly bug-for-bug compatible replacements are
`RectBivariateSpline` on regular grids, and `bisplrep`/`bisplev` for
scattered 2D data.

In new code, for regular grids use `RegularGridInterpolator` instead.
For scattered data, prefer `LinearNDInterpolator` or
`CloughTocher2DInterpolator`.

For more details see
`https://scipy.github.io/devdocs/notebooks/interp_transition_guide.html`

  f_u = interp2d(x, y, u, kind='linear')
C:\Users\micha\AppData\Local\Temp\ipykernel_17528\2552484690.py:28: DeprecationWarning: `interp2d` is deprecated!
`interp2d` is deprecated in SciPy 1.10 and will be removed in SciPy 1.13.0.

For legacy code, nearly bug-for-bug compatible replacements are
`RectBivariateSpline` on regular grids, and `bisplrep`/`bisplev` for
scattered 2D data.

In new code, for regul